# Compton Analysis Notebook

In [ ]:
ANGLES_OUTPUT_PATH = r"G:\.shortcut-targets-by-id\1IcR20O5deY8V-HZhJrHf7ZL2cmAiG7GY\ComptonMeasurements\Angles"
MCA_FILE_EXTENSION = ".mca"
MCA_FIRST_NON_ZERO_CHANNEL = 568

%load_ext autoreload
%autoreload 2

In [3]:
import os
import pathlib
import mca_output
import plotly_utils

def find_mca_files_in_folder(folder: str) -> list[pathlib.Path]:
    folder_path = pathlib.Path(folder)
    return [pathlib.Path.joinpath(folder_path, out_file) for out_file in os.listdir(folder) if pathlib.Path(out_file).suffix == MCA_FILE_EXTENSION]

# Find all MCA files in given folder
if not os.path.exists(ANGLES_OUTPUT_PATH) or not os.path.isdir(ANGLES_OUTPUT_PATH):
    raise FileNotFoundError(f"Directory \"{ANGLES_OUTPUT_PATH}\" does not exist")

angle_calibration_mca_files = find_mca_files_in_folder(ANGLES_OUTPUT_PATH)
angle_calibration_mca_outs = [mca_output.parse_output_file(mca_file.absolute()) for mca_file in angle_calibration_mca_files]


In [4]:
for mca_out in angle_calibration_mca_outs:
    fig = plotly_utils.generate_mca_out_figure(mca_out, pathlib.Path(mca_out.path).stem)
    fig.show()

### Extract Maximum Intensity for Each Angle

The following excerpt extracts the bin with the highest count number around the 3000 region.

In [5]:
import numpy as np

for mca_out in angle_calibration_mca_outs:
    channel_count = np.array(mca_out.channel_count_list, np.float32) / mca_out.measurement_time
    max_index = np.argmax(channel_count)
    max_counts = channel_count[max_index]
    print(f"{pathlib.Path(mca_out.path).stem}: channel: {max_index}, counts: {max_counts}")

angle_2.5: channel: 2916, counts: 163.4600067138672
angle_-10: channel: 2844, counts: 12.600000381469727
angle_7.5: channel: 2930, counts: 182.10000610351562
angle_5: channel: 2930, counts: 177.69000244140625
angle_0: channel: 2930, counts: 134.7899932861328
angle_20: channel: 2894, counts: 51.5099983215332
angle_-5: channel: 2891, counts: 64.52999877929688
angle_25: channel: 2870, counts: 6.869999885559082
angle_15: channel: 2912, counts: 124.7300033569336
angle_10: channel: 2920, counts: 173.4499969482422
angle_-15: channel: 2833, counts: 0.9300000071525574


We can see from the above that indeed at the angle 7.5 degrees the most counts per unit time (100s live time) is measured at around bin 2930.

### Fitting a Gaussians for Compton Spectrum & Gamma Decay Peak

In [6]:
import typing
import curve_fitter
import analysis_utils

filter_7_5_measurement: typing.Callable[[mca_output.MCAOutput], bool] = lambda mca_out: pathlib.Path(mca_out.path).stem == "angle_7.5"
targetless_7_5_measurement = next((mca_out for mca_out in angle_calibration_mca_outs if filter_7_5_measurement(mca_out)), None)

if not targetless_7_5_measurement:
    raise RuntimeError("Measurment not found")

channel_counts = np.array(targetless_7_5_measurement.channel_count_list) / targetless_7_5_measurement.measurement_time
channel_counts_uncertainty = np.sqrt(channel_counts)

MEASUREMENT_LENGTH = len(channel_counts)

channel_index = np.arange(MEASUREMENT_LENGTH, dtype=np.float32)
channel_uncertainty = np.full([MEASUREMENT_LENGTH], 1./np.sqrt(3))

gaussian_peak_model_data = curve_fitter.ModelData(channel_index[2930-200:2930+200], channel_counts_uncertainty[2930-200:2930+200], channel_counts[2930-200:2930+200], channel_counts_uncertainty[2930-200:2930+200])

beta, sd, chi_sq, p_value = curve_fitter.odr_fit_gaussian(gaussian_peak_model_data, [2930, 100, 40000])
initial_guess_model = curve_fitter.vectorized_gaussian(2930, 100, 40000)
initial_guess_data = initial_guess_model(np.arange(MEASUREMENT_LENGTH, dtype=np.float32))
gaussian_peak_odr_data = curve_fitter.vectorized_gaussian(beta[0], beta[1], beta[2])(np.arange(MEASUREMENT_LENGTH, dtype=np.float32))
print("Gaussian Peak fitting data: ", beta, sd, chi_sq, p_value, "\n")

compton_spectrum_model_data = curve_fitter.ModelData(channel_index[600:2400], channel_counts_uncertainty[600:2400], channel_counts[600:2400], channel_counts_uncertainty[600:2400])
first_gaussian_initial_guess = [900 , 380, np.sum(channel_counts[735:925])*2]
second_gaussian_initial_guess = [1980, 360, np.sum(channel_counts[1900:2080])*2]

print(f"Normalization guesses: {np.sum(channel_counts[735:925])*2}, {np.sum(channel_counts[1900:2080])*2}")
beta, sd, chi_sq, p_value = curve_fitter.odr_fit_two_gaussian_sum(compton_spectrum_model_data, first_gaussian_initial_guess+second_gaussian_initial_guess)
print(f"Compton spectrum fitting data:\nFirst gaussian: values: {beta[0:3]}, errors: {sd[0:3]}\nSecond gaussian: values: {beta[3:]}, errors: {sd[3:]}\nChi-sq: {chi_sq}, p-value: {p_value}")
compton_spectrum_odr_data = curve_fitter.vectorized_two_gaussian_sum(beta)(np.arange(MEASUREMENT_LENGTH, dtype=np.float32))

fig = plotly_utils.generate_scatter_and_line_plot(channel_counts, gaussian_peak_odr_data, compton_spectrum_odr_data)
fig.update_layout(height=800)
fig.show()

background_noise, noise_uncertainty = analysis_utils.find_background_noise(channel_counts[4200:6100])

Gaussian Peak fitting data:  [ 2927.54656851   104.93651132 46784.93067335] [ 0.11047535  0.10303988 42.759325  ] 0.02108790325893657 1.0 

Normalization guesses: 5435.26, 5634.0
Compton spectrum fitting data:
First gaussian: values: [  984.3736844    355.7771356  13854.53620126], errors: [  4.27457214   5.06524425 213.75229392]
Second gaussian: values: [ 1829.36709104   318.96962047 13092.08414216], errors: [  4.29904569   2.71489828 179.92232315]
Chi-sq: 0.058869743912641125, p-value: 1.0


In [ ]:
first_compton_gaussian_guess = curve_fitter.GaussianFittingParameters(900 , 300, np.sum(channel_counts[735:925])*4)
second_compton_gaussian_guess = curve_fitter.GaussianFittingParameters(2000, 300, np.sum(channel_counts[1900:2080])*4)
main_peak_guess = curve_fitter.GaussianFittingParameters(2930, 100, 48000)

gaussian_sum_model_data = curve_fitter.ModelData(channel_index[MCA_FIRST_NON_ZERO_CHANNEL:5000], channel_uncertainty[MCA_FIRST_NON_ZERO_CHANNEL:5000], channel_counts[MCA_FIRST_NON_ZERO_CHANNEL:5000], channel_counts_uncertainty[MCA_FIRST_NON_ZERO_CHANNEL:])
fitting_datas, uncertainties, chi_sq, p_value  = curve_fitter.odr_fit_gaussian_sum(gaussian_sum_model_data, [first_compton_gaussian_guess, second_compton_gaussian_guess, main_peak_guess])

for i, (gaussian_fit, uncertainty) in enumerate(zip(fitting_datas, uncertainties)):
    print(f"Gaussian number #{i+1}: Fitting Params: {gaussian_fit}, Uncertainties: {uncertainty}")

print(f"Chi sq: {chi_sq}, P-Value: {p_value}")

fitted_line_data = curve_fitter.vectorized_gaussian_sum(fitting_datas)(channel_index)
main_peak_data = curve_fitter.vectorized_gaussian(fitting_datas[2].mean, fitting_datas[2].std_dev, fitting_datas[2].normalization)(channel_index)
fig = plotly_utils.generate_scatter_and_line_plot(channel_counts, fitted_line_data)

fig.update_layout(height=800)
fig.show()

Gaussian number #1: Fitting Params: GaussianFittingParameters(mean=np.float64(878.6334599672823), std_dev=np.float64(280.4092491767224), normalization=np.float64(8896.743805240487)), Uncertainties: (np.float64(14.897857903438455), np.float64(16.834732366480313), np.float64(821.5545414505129))
Gaussian number #2: Fitting Params: GaussianFittingParameters(mean=np.float64(1716.9932106354372), std_dev=np.float64(439.5359197873328), normalization=np.float64(18323.4187241818)), Uncertainties: (np.float64(21.756779321582663), np.float64(15.123321472845646), np.float64(761.8749348659959))
Gaussian number #3: Fitting Params: GaussianFittingParameters(mean=np.float64(2930.484958180282), std_dev=np.float64(111.26973995184248), normalization=np.float64(47298.48898279369)), Uncertainties: (np.float64(0.6496829520495515), np.float64(0.5269060750640636), np.float64(272.2705502733638))
Chi sq: 1.5138699307522738, P-Value: 1.0


In [ ]:
find_mca_files_in_folder()